# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [21]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''
import pandas as pd
reviews = pd.read_csv('IMDB Dataset.csv')
reviews = reviews.truncate(after = 599)

reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
595,I read the book before watching the movie and ...,positive
596,"Years ago, Sara, a young girl witnessed her pa...",negative
597,In short if you want to watch Burt Reynolds be...,positive
598,I have watched Love Jones over thirty times. I...,positive


In [22]:
#Split the data into train and split
import numpy as np
from sklearn.model_selection import train_test_split
X=reviews.review
y=reviews.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=20)


In [23]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0
train_reviews = pd.DataFrame(data=X_train, columns=['review'])
test_reviews = pd.DataFrame(data=X_test, columns=['review'])
train_sentiment= pd.DataFrame(data=y_train, columns=['sentiment'])
test_sentiment = pd.DataFrame(data=y_test, columns=['sentiment'])


train_sentiment['sentiment']=train_sentiment['sentiment'].replace(['positive'],1)
train_sentiment['sentiment']=train_sentiment['sentiment'].replace(['negative'],0)
test_sentiment['sentiment']=test_sentiment['sentiment'].replace(['positive'],1)
test_sentiment['sentiment']=test_sentiment['sentiment'].replace(['negative'],0)

test_sentiment

,sentiment
493,1
172,1
352,0
506,0
595,1
...,...
508,0
476,1
475,1
235,1


In [24]:
train_reviews.head()

,review
325,Although most Americans have little knowledge ...
553,This movie pretty much sucked. I'm in the Army...
582,I would recommend this film to anyone who is s...
488,A fey story of a Martian attempt to colonize E...
496,"First off, I have to say that I loved the book..."


## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [27]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''
import nltk
from nltk.tokenize import word_tokenize
train_reviews['Tokenized_Review'] = train_reviews['review'].str.lower().apply(word_tokenize)
train_reviews[['review','Tokenized_Review']][:10]

,review,Tokenized_Review
325,Although most Americans have little knowledge ...,"[although, most, americans, have, little, know..."
553,This movie pretty much sucked. I'm in the Army...,"[this, movie, pretty, much, sucked, ., i, 'm, ..."
582,I would recommend this film to anyone who is s...,"[i, would, recommend, this, film, to, anyone, ..."
488,A fey story of a Martian attempt to colonize E...,"[a, fey, story, of, a, martian, attempt, to, c..."
496,"First off, I have to say that I loved the book...","[first, off, ,, i, have, to, say, that, i, lov..."
76,The Last Hard Men finds James Coburn an outlaw...,"[the, last, hard, men, finds, james, coburn, a..."
427,This is an OK adaptation of the breath taking ...,"[this, is, an, ok, adaptation, of, the, breath..."
550,A response to previous comments made by reside...,"[a, response, to, previous, comments, made, by..."
329,What a surprisingly good movie this one turned...,"[what, a, surprisingly, good, movie, this, one..."
260,Paul Verhoeven's predecessor to his breakout h...,"[paul, verhoeven, 's, predecessor, to, his, br..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [28]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle

def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    for seq in token_seqs:
        for token in seq:
            if token in token_counts:
                token_counts[token] += 1
            else:
                token_counts[token] = 1
    

    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
    
    lexicon =  [token for token, count in token_counts.items() if count >= min_freq]
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    lexicon = {token:idx + 2 for idx,token in enumerate(lexicon)}
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)
    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)




{'although': 2, 'most': 3, 'americans': 4, 'have': 5, 'little': 6, 'knowledge': 7, 'of': 8, 'his': 9, 'work': 10, 'other': 11, 'than': 12, 'star': 13, 'wars': 14, ',': 15, 'alec': 16, 'guinness': 17, 'produced': 18, 'an': 19, 'amazing': 20, 'body': 21, '--': 22, 'particularly': 23, 'in': 24, 'the': 25, '1940s-1950s': 26, 'ranging': 27, 'from': 28, 'dramas': 29, 'to': 30, 'quirky': 31, 'comedies': 32, '.': 33, 'i': 34, 'love': 35, 'as': 36, 'they': 37, 'are': 38, 'so': 39, 'well-done': 40, 'and': 41, 'seem': 42, 'natural': 43, 'real': 44, 'on': 45, 'screen': 46, 'far': 47, 'different': 48, 'usual': 49, 'fare': 50, 'hollywood.': 51, '<': 52, 'br': 53, '/': 54, '>': 55, 'this': 56, 'being': 57, 'said': 58, 'was': 59, 'film': 60, 'that': 61, 'sparked': 62, 'my': 63, 'interest': 64, 'these': 65, 'movies': 66, 'it': 67, "'s": 68, 'plot': 69, 'odd': 70, 'cute': 71, 'is': 72, 'very': 73, 'unlikely': 74, 'would': 75, 'been': 76, 'made': 77, 'anywhere': 78, 'except': 79, 'for': 80, 'ealing': 81,

###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [29]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in token_seq]  for token_seq in token_seqs]
    return idx_seqs

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

,Tokenized_Review,Review_Idxs
325,"[although, most, americans, have, little, know...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
553,"[this, movie, pretty, much, sucked, ., i, 'm, ...","[56, 164, 165, 166, 167, 33, 34, 168, 24, 25, ..."
582,"[i, would, recommend, this, film, to, anyone, ...","[34, 75, 204, 56, 60, 30, 214, 215, 72, 216, 8..."
488,"[a, fey, story, of, a, martian, attempt, to, c...","[85, 250, 251, 8, 85, 252, 253, 30, 254, 255, ..."
496,"[first, off, ,, i, have, to, say, that, i, lov...","[213, 149, 15, 34, 5, 30, 196, 61, 34, 412, 25..."
76,"[the, last, hard, men, finds, james, coburn, a...","[25, 466, 467, 450, 468, 469, 470, 19, 471, 47..."
427,"[this, is, an, ok, adaptation, of, the, breath...","[56, 72, 19, 585, 586, 8, 25, 587, 540, 413, 8..."
550,"[a, response, to, previous, comments, made, by...","[85, 627, 30, 628, 629, 77, 338, 630, 8, 25, 6..."
329,"[what, a, surprisingly, good, movie, this, one...","[192, 85, 774, 591, 164, 56, 123, 775, 428, 30..."
260,"[paul, verhoeven, 's, predecessor, to, his, br...","[788, 789, 68, 790, 30, 9, 791, 792, 793, 794,..."


In [30]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow = numpy.array([numpy.bincount(numpy.array(idx_seq), minlength=matrix_length) 
                            for idx_seq in idx_seqs])
    return bow
    


bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1) #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pd.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

TRAIN INPUT:
 [[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
SHAPE: (420, 12214) 



,Word,Count
0,,0
1,<UNK>,0
2,although,1
3,most,1
4,americans,1
...,...,...
12209,slayer,0
12210,toons,0
12211,rental,0
12212,pumpkin,0


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [31]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    #find the biggest review's length and save it in the variable below
    max_seq_len =max([len(idx_seq) for idx_seq in idx_seqs])
    pad = pad_sequences(sequences=idx_seqs, maxlen=max_seq_len)
    return pad
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    
    


train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[    0     0     0 ...    24   163    33]
 [    0     0     0 ...    56   123    33]
 [    0     0     0 ...    91    67   112]
 ...
 [    0     0     0 ...   420  5451   112]
 [    0     0     0 ... 11305   428    33]
 [    0     0     0 ...   164  1506    33]]
SHAPE: (420, 1506) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [32]:
'''Create the model'''

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    input_layer = Input(shape=(None,))
    
    # Layer 2
    embedding_layer = Embedding(input_dim=n_input_nodes,output_dim=n_embedding_nodes,mask_zero=True)(input_layer)
    
    # Layer 3
    gru_layer = GRU(units=n_hidden_nodes)(embedding_layer)
    # Layer 4
    output_layer = Dense(units=1)(gru_layer)
    #Specify which layers are input and output, compile model with loss and optimization functions
    model = Model(inputs=[input_layer], outputs=output_layer)
    model.compile(loss="mean_squared_error", optimizer='adam')
    
    return model


In [33]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

###  <font color='#6629b2'>Training</font>

In [34]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''
tensor=tf.convert_to_tensor(train_sentiment['sentiment'])
rnn_model.fit(x=train_padded_idxs, y=tensor, batch_size=20, epochs=5)



Epoch 1/5
21/21 [==============================] - 251s 12s/step - loss: 0.2711
Epoch 2/5
21/21 [==============================] - 280s 13s/step - loss: 0.1429
Epoch 3/5
21/21 [==============================] - 280s 13s/step - loss: 0.0621
Epoch 4/5
21/21 [==============================] - 283s 13s/step - loss: 0.0410
Epoch 5/5
21/21 [==============================] - 302s 14s/step - loss: 0.0208


###  <font color='#6629b2'>Prediction</font>

In [35]:
'''Put test reviews in padded matrix just how we did for train_reviews'''
test_reviews['Tokenized_Review'] = test_reviews['review'].str.lower().apply(word_tokenize)
test_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=test_reviews['Tokenized_Review'],lexicon=lexicon)
test_padded_idxs = pad_idx_seqs(test_reviews['Review_Idxs'])

print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

TEST INPUT:
 [[   0    0    0 ...   30  625   33]
 [   0    0    0 ...  193  287   33]
 [   0    0    0 ...  351 3807  128]
 ...
 [   0    0    0 ...  173 1469   33]
 [   0    0    0 ...  524  601    1]
 [   0    0    0 ... 1940    1   33]]
SHAPE: (180, 1218) 



In [36]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = numpy.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

###  <font color='#6629b2'>Evaluation</font>

In [40]:
'''Evaluate the model with R^2'''

# print the r2 score
from sklearn.metrics import r2_score 
r2Score = r2_score(y_true=test_sentiment, y_pred=test_reviews['RNN_Pred_Rating'])
print('R2 score=',round(r2Score,6))

R2 score= -0.550042


On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [41]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

EMBEDDING LAYER OUTPUT SHAPE: (180, 1218, 300)
[[ 0.02646316  0.04013968  0.0469967  ...  0.03789624 -0.0152205
   0.03878452]
 [ 0.02646316  0.04013968  0.0469967  ...  0.03789624 -0.0152205
   0.03878452]
 [ 0.02646316  0.04013968  0.0469967  ...  0.03789624 -0.0152205
   0.03878452]
 ...
 [-0.05423133 -0.03326934  0.03916905 ...  0.00944157 -0.04604006
  -0.03263585]
 [-0.01048676  0.00841777 -0.00885442 ... -0.01910497  0.00921863
   0.06074259]
 [ 0.00656235  0.01602003  0.02756088 ...  0.00798869 -0.0026723
   0.02253819]]


## <font color='#6629b2'>Conclusion</font>

As mentioned above, the models shown here could be applied to any task where the goal is to predict a score for a particular sequence. For ratings prediction, this output is ordinal, but it could also be categorical with a few simple changes to the output layer of the model. 

## <font color='#6629b2'>More resources</font>

Yoav Goldberg's book [Neural Network Methods for Natural Language Processing](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037) is a thorough introduction to neural networks for NLP tasks in general.

If you'd like to learn more about what Keras is doing under the hood, there is a [Theano tutorial](http://deeplearning.net/tutorial/lstm.html) that also applies an RNN to sentiment prediction, using the same dataset here

Andrej Karpathy's blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is very helpful for understanding the mathematical details of an RNN, applied to the task of language modeling. It also provides raw Python code with an implementation of the backpropagation algorithm.

TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

Chris Olah provides a good [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTM RNNs work (this explanation also applies to the GRU model used here)

Denny Britz's [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) documents well both the technical details of RNNs and their implementation in Python.